# 第14章：Multi-Agent系统

## 学习目标

本章将学习：
1. Multi-Agent系统的核心概念
2. Subagents模式（Supervisor/主从架构）
3. Handoffs模式（转移控制）
4. Router模式（路由并行）
5. Agent间通信机制
6. 模式选择和对比
7. 实战：多Agent协作系统

## 为什么需要Multi-Agent？

**单Agent的局限**：
- 工具太多导致选择困难
- Context窗口容易爆满
- 专业领域知识难以整合
- 无法并行处理子任务

**Multi-Agent的优势**：
1. **Context管理**：每个Agent维护独立上下文，避免信息过载
2. **专业化分工**：每个Agent专注特定领域，提高准确性
3. **并行处理**：多个Agent同时工作，提升效率
4. **模块化开发**：不同团队独立开发和维护各自的Agent

---

## Multi-Agent架构模式对比

| 模式 | 工作原理 | 适用场景 |
|------|---------|---------|
| **Subagents** | 主Agent将子Agent作为工具调用 | 中心化控制、多领域协作 |
| **Handoffs** | Agent间通过工具调用转移控制 | 有状态的工作流、顺序处理 |
| **Router** | 路由层分类后并行调用多Agent | 多数据源查询、并行任务 |
| **Skills** | 单Agent按需加载专门知识 | 动态知识加载 |

---

## 核心概念

### Context隔离

```
单Agent（Context混杂）:
User → Agent[所有工具+所有历史] → Response
        ↑ 10,000+ tokens

Multi-Agent（Context隔离）:
User → Main Agent → SubAgent A[专门工具+独立上下文] → Result
                  → SubAgent B[专门工具+独立上下文] → Result
         ↑ 3,000 tokens     ↑ 2,000 tokens
```

In [1]:
# 环境配置
import os
import sys

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.agents import create_agent

# 初始化模型
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL,
)

print("环境配置完成！")
print(f"模型: {model.model_name}")

环境配置完成！
模型: gpt-4.1-mini


## 1. Subagents模式（Supervisor/主从）

### 工作原理

```
User
  ↓
Main Agent (Supervisor)
  ├→ 调用SubAgent A（作为工具）
  ├→ 调用SubAgent B（作为工具）
  └→ 综合结果回复User
```

### 核心特点

- **中心化控制**：Main Agent决定何时调用哪个SubAgent
- **Context隔离**：SubAgent有独立的上下文
- **无状态**：SubAgent不记忆历史，每次调用都是新的
- **工具化**：SubAgent被包装成工具给Main Agent使用

### 适用场景

- 有明确的领域划分（日历、邮件、CRM等）
- 需要中心化的工作流控制
- SubAgent不需要直接与用户对话

In [2]:
print("【演示：Subagents模式】")
print()

# 步骤1：创建专门的SubAgents

# 日历Agent的工具
@tool
def check_calendar(date: str) -> str:
    """查询指定日期的日程安排"""
    schedules = {
        "2024-03-15": "上午10点：技术评审会议",
        "2024-03-16": "下午2点：客户演示",
        "2024-03-17": "全天：团队建设活动"
    }
    return schedules.get(date, "当天无安排")

@tool
def add_event(date: str, event: str) -> str:
    """添加新的日程安排"""
    return f"已添加事件：{date} - {event}"

# 创建Calendar SubAgent
calendar_agent = create_agent(
    model=model,
    tools=[check_calendar, add_event],
    system_prompt="你是日历助手，专门处理日程查询和安排。"
)

# 邮件Agent的工具
@tool
def search_emails(keyword: str) -> str:
    """搜索邮件"""
    emails = {
        "会议": "找到3封关于会议的邮件",
        "报告": "找到5封关于报告的邮件",
        "审批": "找到2封待审批邮件"
    }
    return emails.get(keyword, f"未找到关于'{keyword}'的邮件")

@tool
def send_email(to: str, subject: str) -> str:
    """发送邮件"""
    return f"已发送邮件给{to}，主题：{subject}"

# 创建Email SubAgent
email_agent = create_agent(
    model=model,
    tools=[search_emails, send_email],
    system_prompt="你是邮件助手，专门处理邮件查询和发送。"
)

print("✓ SubAgents创建完成")
print("  - Calendar Agent: 处理日程相关")
print("  - Email Agent: 处理邮件相关")

【演示：Subagents模式】

✓ SubAgents创建完成
  - Calendar Agent: 处理日程相关
  - Email Agent: 处理邮件相关


### 1.2 将SubAgent包装成工具

In [3]:
print("【将SubAgent包装成工具】")
print()

# 包装Calendar Agent为工具
@tool
def calendar_assistant(request: str) -> str:
    """处理日历相关请求，如查询日程、添加事件等"""
    response = calendar_agent.invoke({"messages": [{"role": "user", "content": request}]})
    return response['messages'][-1].content

# 包装Email Agent为工具
@tool
def email_assistant(request: str) -> str:
    """处理邮件相关请求，如搜索邮件、发送邮件等"""
    response = email_agent.invoke({"messages": [{"role": "user", "content": request}]})
    return response['messages'][-1].content

print("✓ SubAgent工具包装完成")
print("  - calendar_assistant: 调用Calendar Agent")
print("  - email_assistant: 调用Email Agent")

【将SubAgent包装成工具】

✓ SubAgent工具包装完成
  - calendar_assistant: 调用Calendar Agent
  - email_assistant: 调用Email Agent


### 1.3 创建Supervisor（Main Agent）

In [4]:
print("【创建Supervisor Agent】")
print()

# 创建Main Agent（Supervisor）
supervisor = create_agent(
    model=model,
    tools=[calendar_assistant, email_assistant],
    system_prompt="""你是个人助理Supervisor，协调日历和邮件两个专门助手。
    
根据用户请求的类型，选择合适的助手：
- 日程、会议、安排 → 使用calendar_assistant
- 邮件、发送、查找 → 使用email_assistant
- 如果涉及多个领域，可以依次调用多个助手

综合各助手的结果，给用户清晰的回复。"""
)

print("✓ Supervisor Agent创建完成")
print()

# 测试场景
test_queries = [
    "查询我3月15日的日程",
    "搜索关于会议的邮件",
    "帮我查一下明天的安排，然后给张三发邮件通知会议时间"
]

for i, query in enumerate(test_queries, 1):
    print(f"{'='*60}")
    print(f"【测试 {i}】: {query}")
    print('-'*60)
    
    response = supervisor.invoke({"messages": [{"role": "user", "content": query}]})
    
    # 显示调用了哪些SubAgent
    print("\n工具调用链：")
    for msg in response['messages']:
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            for tc in msg.tool_calls:
                print(f"  → Supervisor调用: {tc['name']}")
    
    print(f"\n最终回复: {response['messages'][-1].content}")
    print()

print("="*60)
print("说明: Supervisor根据请求类型自动选择合适的SubAgent")

【创建Supervisor Agent】

✓ Supervisor Agent创建完成

【测试 1】: 查询我3月15日的日程
------------------------------------------------------------

工具调用链：
  → Supervisor调用: calendar_assistant

最终回复: 您3月15日当天没有安排的日程。需要我帮您添加新的日程吗？

【测试 2】: 搜索关于会议的邮件
------------------------------------------------------------

工具调用链：
  → Supervisor调用: email_assistant

最终回复: 我找到了3封关于会议的邮件，您需要查看其中的哪一封，或者需要我帮您做其他操作？

【测试 3】: 帮我查一下明天的安排，然后给张三发邮件通知会议时间
------------------------------------------------------------

工具调用链：
  → Supervisor调用: calendar_assistant
  → Supervisor调用: email_assistant

最终回复: 明天（2024年4月28日）您没有安排任何日程。如果需要，我可以帮您添加新的日程安排。

另外，已经给张三发送了主题为“会议时间通知”的邮件，内容是通知会议时间，具体时间待确认。

请问还有其他需要帮忙的吗？

说明: Supervisor根据请求类型自动选择合适的SubAgent


## 2. Handoffs模式（状态转移）

### 工作原理

```
User → Agent A → [调用transfer_to_B工具] → Agent B → [调用transfer_to_C工具] → Agent C
         ↓                                    ↓                                   ↓
      State更新                           State更新                           State更新
```

### 核心机制

- **工具触发转移**：通过调用`transfer_to_X`工具切换Agent
- **状态驱动**：维护`current_agent`或`current_step`状态变量
- **持久化上下文**：对话历史跨Agent保留
- **Middleware配置**：根据状态动态调整system_prompt和tools

### 适用场景

- 有明确的工作流状态（客服：收集信息→分类→解决）
- Agent需要与用户直接对话
- 顺序处理，非并行

### 对比Subagents

| 特性 | Subagents | Handoffs |
|------|-----------|----------|
| 上下文 | 隔离，每次调用新建 | 共享，跨Agent保留 |
| 控制 | 中心化（Supervisor决定） | 去中心化（Agent自主转移） |
| 用户交互 | 仅Supervisor与用户对话 | 各Agent都可与用户对话 |
| 适合场景 | 并行任务、多领域 | 顺序工作流、有状态 |

In [5]:
print("【演示：Handoffs模式（简化）】")
print()
print("Handoffs模式的完整实现需要LangGraph的State管理和Middleware。")
print("这里演示核心概念：通过状态变量和工具实现Agent间转移。")
print()

from typing import TypedDict, Literal
from pydantic import BaseModel, Field

# 定义状态
class SupportState(TypedDict):
    current_step: Literal["warranty", "classify", "resolve"]
    warranty_status: str | None
    issue_type: str | None

# 定义转移工具
@tool
def record_warranty_status(status: str) -> str:
    """记录保修状态（in_warranty或out_of_warranty）并转到下一步"""
    # 在实际应用中，这会更新State并触发到下一个阶段
    return f"保修状态已记录：{status}，准备分类问题类型"

@tool
def record_issue_type(issue_type: str) -> str:
    """记录问题类型（hardware或software）并转到解决阶段"""
    return f"问题类型已记录：{issue_type}，准备提供解决方案"

@tool
def provide_solution(solution: str) -> str:
    """提供解决方案"""
    return f"解决方案：{solution}"

# 创建客服Agent（模拟Handoffs）
support_agent = create_agent(
    model=model,
    tools=[record_warranty_status, record_issue_type, provide_solution],
    system_prompt="""你是客服支持Agent。处理流程：
    
1. 先询问设备是否在保修期内，使用record_warranty_status记录
2. 然后询问是硬件还是软件问题，使用record_issue_type记录
3. 最后使用provide_solution提供解决方案

每个阶段收集信息后，调用对应工具记录并转到下一阶段。"""
)

print("【模拟对话流程】")
conversation = [
    {"role": "user", "content": "我的手机屏幕坏了"},
]

# 第1轮：Agent询问保修状态
print("\n第1轮对话：")
response1 = support_agent.invoke({"messages": conversation})
print(f"Agent: {response1['messages'][-1].content}")

# 用户回复
conversation.extend(response1['messages'])
conversation.append({"role": "user", "content": "是的，还在保修期内"})

# 第2轮：Agent记录状态并询问问题类型
print("\n第2轮对话：")
response2 = support_agent.invoke({"messages": conversation})
print(f"Agent: {response2['messages'][-1].content}")

print("\n说明: 这是Handoffs模式的简化演示。")
print("完整实现需要：")
print("  1. LangGraph的State管理")
print("  2. Middleware根据current_step动态配置Agent")
print("  3. Checkpointer持久化状态")
print("\n参考官方文档的Customer Support教程了解完整实现")

【演示：Handoffs模式（简化）】

Handoffs模式的完整实现需要LangGraph的State管理和Middleware。
这里演示核心概念：通过状态变量和工具实现Agent间转移。

【模拟对话流程】

第1轮对话：
Agent: 请问您的手机目前是否还在保修期内？

第2轮对话：
Agent: 感谢确认。请问您的问题是硬件问题还是软件问题？

说明: 这是Handoffs模式的简化演示。
完整实现需要：
  1. LangGraph的State管理
  2. Middleware根据current_step动态配置Agent
  3. Checkpointer持久化状态

参考官方文档的Customer Support教程了解完整实现


## 3. Router模式（路由并行）

### 工作原理

```
User Query
    ↓
Router（分类+路由）
    ├→ Agent A（并行）→ Result A
    ├→ Agent B（并行）→ Result B
    └→ Agent C（并行）→ Result C
              ↓
        Synthesizer（综合）
              ↓
          Final Answer
```

### 核心特点

- **显式路由**：有专门的路由步骤分类query
- **并行执行**：多个Agent同时工作
- **结果综合**：最后统一整合各Agent的输出
- **多数据源**：适合跨多个知识域查询

### 适用场景

- 多数据源查询（GitHub+Notion+Slack）
- 需要并行提速
- 领域边界清晰

In [6]:
print("【演示：Router模式】")
print()

from typing import List
from pydantic import BaseModel

# 步骤1：创建专门领域的Agents
@tool
def search_python_docs(query: str) -> str:
    """搜索Python文档"""
    docs = {
        "语法": "Python使用缩进表示代码块，支持动态类型",
        "性能": "Python是解释型语言，性能较C++慢，但开发效率高",
        "应用": "Python广泛应用于Web开发、数据科学、AI等领域"
    }
    return docs.get(query, f"Python文档：{query}相关内容")

python_agent = create_agent(
    model=model,
    tools=[search_python_docs],
    system_prompt="你是Python专家，回答Python相关问题。"
)

@tool
def search_javascript_docs(query: str) -> str:
    """搜索JavaScript文档"""
    docs = {
        "语法": "JavaScript是动态类型语言，使用大括号表示代码块",
        "性能": "JavaScript通过V8引擎编译，性能较好",
        "应用": "JavaScript主要用于Web前端开发，也可用于Node.js后端"
    }
    return docs.get(query, f"JavaScript文档：{query}相关内容")

javascript_agent = create_agent(
    model=model,
    tools=[search_javascript_docs],
    system_prompt="你是JavaScript专家，回答JavaScript相关问题。"
)

@tool
def search_rust_docs(query: str) -> str:
    """搜索Rust文档"""
    docs = {
        "语法": "Rust是静态类型语言，有严格的所有权系统",
        "性能": "Rust性能接近C++，且内存安全",
        "应用": "Rust用于系统编程、嵌入式、高性能服务"
    }
    return docs.get(query, f"Rust文档：{query}相关内容")

rust_agent = create_agent(
    model=model,
    tools=[search_rust_docs],
    system_prompt="你是Rust专家，回答Rust相关问题。"
)

print("✓ 创建了3个领域专家Agent")

【演示：Router模式】

✓ 创建了3个领域专家Agent


### 3.2 实现Router和Synthesizer

In [7]:
print("【Router + 并行查询 + Synthesizer】")
print()

# 步骤2：创建Router（分类查询）
class QueryRouting(BaseModel):
    """查询路由结果"""
    python_query: str | None = Field(default=None, description="发给Python Agent的查询，如果不相关则为None")
    javascript_query: str | None = Field(default=None, description="发给JavaScript Agent的查询，如果不相关则为None")
    rust_query: str | None = Field(default=None, description="发给Rust Agent的查询，如果不相关则为None")

router_agent = create_agent(
    model=model,
    tools=[],
    response_format=QueryRouting,
    system_prompt="""分析用户query，将其分解为针对不同语言的子查询。
    
如果query涉及某个语言，为该语言生成子查询；如果不涉及，设为None。
例如："比较Python和JavaScript的性能" 
→ python_query="性能", javascript_query="性能", rust_query=None"""
)

# 步骤3：并行调用多个Agents
def route_and_query(user_query: str) -> dict:
    """路由查询并并行调用相关Agents"""
    print(f"用户查询: {user_query}")
    print()
    
    # 路由
    routing_response = router_agent.invoke({"messages": [{"role": "user", "content": user_query}]})
    routing = routing_response['structured_response']
    
    print("路由结果：")
    if routing.python_query:
        print(f"  → Python: {routing.python_query}")
    if routing.javascript_query:
        print(f"  → JavaScript: {routing.javascript_query}")
    if routing.rust_query:
        print(f"  → Rust: {routing.rust_query}")
    print()
    
    # 并行调用（简化版，实际可用asyncio）
    results = {}
    
    if routing.python_query:
        resp = python_agent.invoke({"messages": [{"role": "user", "content": routing.python_query}]})
        results['python'] = resp['messages'][-1].content
    
    if routing.javascript_query:
        resp = javascript_agent.invoke({"messages": [{"role": "user", "content": routing.javascript_query}]})
        results['javascript'] = resp['messages'][-1].content
    
    if routing.rust_query:
        resp = rust_agent.invoke({"messages": [{"role": "user", "content": routing.rust_query}]})
        results['rust'] = resp['messages'][-1].content
    
    return results

# 步骤4：Synthesizer综合结果
synthesizer_agent = create_agent(
    model=model,
    tools=[],
    system_prompt="根据各语言专家的回答，综合成一个连贯的答案。"
)

def synthesize_results(user_query: str, results: dict) -> str:
    """综合多个Agent的结果"""
    context = "\n\n".join([f"{lang}专家: {answer}" for lang, answer in results.items()])
    prompt = f"用户问题: {user_query}\n\n专家回答:\n{context}\n\n请综合以上信息，给出连贯的答案。"
    
    response = synthesizer_agent.invoke({"messages": [{"role": "user", "content": prompt}]})
    return response['messages'][-1].content

# 测试
print("="*60)
print("【测试Router模式】")
print("="*60)
print()

query = "比较Python、JavaScript和Rust的性能特点"

# 路由并查询
results = route_and_query(query)

print("Agent返回结果：")
for lang, result in results.items():
    print(f"\n[{lang}]\n{result}")

# 综合
print("\n" + "="*60)
print("【综合答案】")
print("="*60)
final_answer = synthesize_results(query, results)
print(final_answer)
print()
print("说明: Router模式实现了并行查询多个领域，然后综合结果")

【Router + 并行查询 + Synthesizer】

【测试Router模式】

用户查询: 比较Python、JavaScript和Rust的性能特点

路由结果：
  → Python: 性能特点
  → JavaScript: 性能特点
  → Rust: 性能特点

Agent返回结果：

[python]
请问您具体是想了解Python的哪些方面的性能特点？比如运行速度、内存使用、多线程性能，还是某个具体模块或功能的性能？这样我可以为您提供更准确的解答。

[javascript]
您说的“性能特点”是指JavaScript的性能特点吗？还是某个具体JavaScript功能或API的性能特点？请您具体说明一下，我好为您提供更准确的解答。

[rust]
Rust的性能特点主要包括以下几个方面：

1. 零成本抽象：Rust的抽象机制在编译时进行优化，不会引入运行时开销，保证高性能。
2. 内存安全且无垃圾回收：通过所有权系统和借用检查，Rust在编译时确保内存安全，避免了垃圾回收带来的性能损耗。
3. 高效的并发支持：Rust提供了安全的并发编程模型，避免数据竞争，提升多线程程序的性能。
4. 静态类型和编译时检查：编译器在编译阶段进行严格的类型检查和错误检测，减少运行时错误，提高执行效率。
5. 直接访问底层硬件：Rust允许直接操作内存和硬件资源，适合系统级编程，性能接近C/C++。

如果你需要更详细的性能优化技巧或具体案例，也可以告诉我。

【综合答案】
Python、JavaScript和Rust在性能特点上各有侧重，适用于不同的应用场景：

1. **Python**  
   Python以易用性和丰富的生态著称，但其运行速度相对较慢，主要因为它是解释型语言，且采用全局解释器锁（GIL）限制了多线程的并发性能。Python的内存使用较为灵活，但在高性能计算或多线程密集型任务中可能表现不佳。通常通过调用C扩展或使用JIT编译器（如PyPy）来提升性能。

2. **JavaScript**  
   JavaScript作为浏览器和服务器端（Node.js）广泛使用的语言，性能依赖于现代引擎（如V8）的优化。它具备较快的启动速度和良好的异步编程模型，适合I/O密集型和事件驱动的应用。JavaScript的性能特点还包括高效的垃圾回收机制

## 4. 模式选择指南

### 效率对比（以多领域查询为例）

假设任务："比较Python、JavaScript、Rust"

| 模式 | 模型调用次数 | Token消耗 | 能否并行 | 适合场景 |
|------|------------|----------|---------|----------|
| **Subagents** | 5次 | ~9K | ✅ 可以 | 多领域协作 |
| **Handoffs** | 7+次 | ~14K+ | ❌ 顺序执行 | 有状态工作流 |
| **Router** | 5次 | ~9K | ✅ 可以 | 多数据源查询 |
| **单Agent** | 1次 | ~15K | ❌ 不适用 | 简单任务 |

---

### 决策树

```
需要Multi-Agent吗？
  ├─ 否 → 使用单Agent
  └─ 是
      ├─ 需要顺序工作流 + 保留上下文？
      │   └─ 是 → Handoffs模式
      ├─ 需要显式路由 + 并行查询多数据源？
      │   └─ 是 → Router模式
      └─ 需要中心化控制 + Context隔离？
          └─ 是 → Subagents模式
```

---

### 使用建议

1. **从单Agent开始**：大多数任务单Agent + 动态工具就足够
2. **Subagents优先**：如果确定需要Multi-Agent，优先考虑Subagents（实现简单，效率高）
3. **Handoffs适合状态机**：客服、审批流程等明确的状态转换场景
4. **Router适合知识库**：多个独立数据源需要并行查询
5. **可以混合**：在LangGraph中组合多种模式

## 5. 实战项目：智能研究助手系统

### 系统需求

构建一个研究助手，支持：
1. **文献搜索**：查找学术论文
2. **数据分析**：处理数据和生成图表
3. **报告生成**：综合信息写报告

### 架构设计：Subagents模式

```
                    Research Supervisor
                            |
         ┌──────────────────┼──────────────────┐
         ↓                  ↓                  ↓
   Literature Agent   Data Agent      Report Agent
   [搜索论文]          [分析数据]       [生成报告]
```

In [8]:
print("【实战项目：智能研究助手系统】")
print()

# 1. 创建文献搜索Agent
@tool
def search_papers(topic: str, year: int | None = None) -> str:
    """搜索学术论文"""
    papers = {
        "机器学习": [
            "Deep Learning (Hinton, 2015)",
            "Attention Is All You Need (Vaswani, 2017)"
        ],
        "RAG": [
            "Retrieval-Augmented Generation (Lewis, 2020)",
            "RAG for Open-Domain QA (Chen, 2021)"
        ]
    }
    results = papers.get(topic, [])
    return f"找到 {len(results)} 篇论文:\n" + "\n".join(results)

@tool
def get_paper_summary(paper_title: str) -> str:
    """获取论文摘要"""
    summaries = {
        "Attention Is All You Need": "提出了Transformer架构，完全基于注意力机制",
        "Retrieval-Augmented Generation": "结合检索和生成，增强LLM的知识能力"
    }
    return summaries.get(paper_title, f"{paper_title}的摘要内容")

literature_agent = create_agent(
    model=model,
    tools=[search_papers, get_paper_summary],
    system_prompt="你是文献搜索专家，帮助用户查找和总结学术论文。"
)

# 2. 创建数据分析Agent
@tool
def analyze_data(data_description: str) -> str:
    """分析数据并生成统计摘要"""
    return f"数据分析结果：均值=85.3, 标准差=12.5, 样本数=100\n趋势：呈上升趋势"

@tool
def create_chart(chart_type: str, data_description: str) -> str:
    """创建图表"""
    return f"已生成{chart_type}图表：{data_description}"

data_agent = create_agent(
    model=model,
    tools=[analyze_data, create_chart],
    system_prompt="你是数据分析专家，处理数据并生成可视化图表。"
)

# 3. 创建报告生成Agent
@tool
def generate_section(section_name: str, content: str) -> str:
    """生成报告章节"""
    return f"## {section_name}\n\n{content}"

@tool
def format_report(sections: str) -> str:
    """格式化完整报告"""
    return f"# 研究报告\n\n{sections}\n\n---\n生成时间：2024-03-15"

report_agent = create_agent(
    model=model,
    tools=[generate_section, format_report],
    system_prompt="你是报告撰写专家，将信息整理成结构化的研究报告。"
)

print("✓ 创建了3个专门Agent：")
print("  - Literature Agent: 文献搜索")
print("  - Data Agent: 数据分析")
print("  - Report Agent: 报告生成")

【实战项目：智能研究助手系统】

✓ 创建了3个专门Agent：
  - Literature Agent: 文献搜索
  - Data Agent: 数据分析
  - Report Agent: 报告生成


### 5.2 组装Research Supervisor

In [9]:
print("【组装Research Supervisor】")
print()

# 将SubAgents包装为工具
@tool
def literature_search(request: str) -> str:
    """搜索学术文献、查找论文、获取论文摘要等"""
    response = literature_agent.invoke({"messages": [{"role": "user", "content": request}]})
    return response['messages'][-1].content

@tool
def data_analysis(request: str) -> str:
    """分析数据、生成统计摘要、创建图表等"""
    response = data_agent.invoke({"messages": [{"role": "user", "content": request}]})
    return response['messages'][-1].content

@tool
def report_writing(request: str) -> str:
    """生成报告、撰写章节、格式化文档等"""
    response = report_agent.invoke({"messages": [{"role": "user", "content": request}]})
    return response['messages'][-1].content

# 创建Supervisor
research_supervisor = create_agent(
    model=model,
    tools=[literature_search, data_analysis, report_writing],
    system_prompt="""你是研究助手Supervisor，协调三个专门助手：
    
- literature_search: 文献搜索和论文查询
- data_analysis: 数据分析和图表生成
- report_writing: 报告撰写和格式化

根据用户的研究需求，选择合适的助手并协调它们完成任务。
对于复杂任务，可以依次调用多个助手。
最后给用户一个完整、连贯的答复。"""
)

print("✓ Research Supervisor创建完成")

【组装Research Supervisor】

✓ Research Supervisor创建完成


### 5.3 测试完整系统

In [10]:
print("【测试智能研究助手系统】")
print()

test_cases = [
    "搜索关于RAG的最新论文",
    "分析我的实验数据：包含100个样本，均值85.3",
    "帮我写一份关于Transformer的研究报告，需要包含文献综述和数据分析"
]

for i, query in enumerate(test_cases, 1):
    print("="*70)
    print(f"【测试 {i}】: {query}")
    print("="*70)
    
    response = research_supervisor.invoke({
        "messages": [{"role": "user", "content": query}]
    })
    
    # 显示SubAgent调用链
    print("\nSubAgent调用链：")
    for msg in response['messages']:
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            for tc in msg.tool_calls:
                print(f"  → {tc['name']}")
    
    print(f"\n最终回复:\n{response['messages'][-1].content}")
    print()

print("="*70)
print("系统特点：")
print("1. Context隔离：每个SubAgent维护独立上下文")
print("2. 专业分工：文献、数据、报告各司其职")
print("3. 灵活编排：Supervisor根据任务动态调用")
print("4. 可扩展：易于添加新的SubAgent")

【测试智能研究助手系统】

【测试 1】: 搜索关于RAG的最新论文

SubAgent调用链：
  → literature_search

最终回复:
目前2024年还没有找到关于Retrieval-Augmented Generation（RAG）的最新学术论文。您是否需要我帮您查找近几年内的相关论文，或者查找相关领域的综述文章？

【测试 2】: 分析我的实验数据：包含100个样本，均值85.3

SubAgent调用链：
  → data_analysis

最终回复:
您的实验数据包含100个样本，均值为85.3，标准差约为12.5，数据呈现上升趋势。统计分析结果显示样本均值85.3，标准差12.5，样本数量100。

建议的图表展示包括：
- 直方图（Histogram）：展示数据分布情况
- 箱线图（Box Plot）：展示中位数、四分位数及异常值
- 折线图（Line Chart）：展示数据趋势变化

如果您需要，我可以帮您生成具体的图表或进行更深入的分析。请告诉我您的具体需求。

【测试 3】: 帮我写一份关于Transformer的研究报告，需要包含文献综述和数据分析

SubAgent调用链：
  → literature_search
  → data_analysis

最终回复:
关于Transformer的研究报告，我已经完成了两个部分的初步工作：

1. 文献综述方面，目前没有找到具体的Transformer相关论文列表。请问您是否希望我尝试用其他关键词或限定年份范围来搜索？或者您需要我总结已有的Transformer模型研究进展和综述？

2. 数据分析方面，我分析了Transformer模型在NLP任务中的性能数据和趋势。结果显示，Transformer模型的平均性能指标（如准确率和F1分数）约为85.3，标准差为12.5，样本数量为100。整体趋势呈上升，说明Transformer模型在NLP任务中的表现逐年提升。我还生成了一张折线图，展示了近年Transformer模型在多个基准测试中的准确率和F1分数的变化趋势，清晰反映了性能提升情况。

请告诉我您是否需要我继续完善文献综述部分，或者需要我提供具体的数据细节和图表，亦或是开始撰写完整的研究报告？

系统特点：
1. Context隔离：每个SubAgent维护独立上下文


## 6. 总结与最佳实践

### 核心要点总结

#### Multi-Agent的本质

Multi-Agent不是为了"多"，而是为了：
1. **Context工程**：合理分配每个Agent看到的信息
2. **专业化**：每个Agent专注一个领域，提高准确性
3. **并行化**：多个Agent同时工作，提升效率
4. **模块化**：独立开发和维护

---

### 三大模式对比

| 维度 | Subagents | Handoffs | Router |
|------|-----------|----------|--------|
| **架构** | 主从式（星型） | 链式（状态机） | 并行式（扇出扇入） |
| **控制** | 中心化 | 去中心化 | 混合式 |
| **上下文** | 隔离 | 共享 | 隔离 |
| **并行** | ✅ 可以 | ❌ 顺序 | ✅ 必须 |
| **用户交互** | 仅Main Agent | 所有Agent | 仅Synthesizer |
| **实现难度** | 简单 | 中等 | 中等 |
| **适合场景** | 多领域协作 | 有状态工作流 | 多数据源查询 |

---

### 何时使用Multi-Agent？

**需要使用**：
- ✅ 单Agent工具太多（20+个）导致选择困难
- ✅ 需要大量领域知识（每个领域2000+ tokens）
- ✅ 需要并行处理多个独立子任务
- ✅ 有明确的顺序约束（Handoffs）

**不需要使用**：
- ❌ 工具数量少（<10个）
- ❌ 任务简单，单Agent足够
- ❌ Context窗口够用
- ❌ 没有并行需求

---

### 最佳实践

#### 1. 从单Agent开始

```python
# 先尝试单Agent + 动态工具
agent = create_agent(
    model=model,
    tools=all_tools,  # 先放一起
    system_prompt="你是多功能助手..."
)

# 如果发现工具选择困难，再拆分成Multi-Agent
```

#### 2. 清晰的领域边界

```python
# ✅ 好的划分：按业务领域
calendar_agent  # 日历相关
email_agent     # 邮件相关
crm_agent       # 客户管理

# ❌ 不好的划分：功能重叠
read_agent      # 只读操作？
write_agent     # 只写操作？
```

#### 3. 详细的工具描述

对于Subagents，工具描述很关键，因为Main Agent通过描述选择SubAgent：

```python
@tool
def calendar_assistant(request: str) -> str:
    """处理日历相关请求，包括：
    - 查询日程安排
    - 添加/修改/删除事件
    - 检查时间冲突
    - 发送会议邀请
    
    适用场景：任何涉及日程、会议、时间安排的请求
    """
    ...
```

#### 4. 独立测试每个Agent

```python
# 先单独测试SubAgent
response = calendar_agent.invoke(...)
assert response正确

# 再测试Main Agent的协调能力
response = supervisor.invoke(...)
```

#### 5. 控制信息流

```python
# 方案1：SubAgent只返回结果（默认）
@tool
def subagent(request: str) -> str:
    response = agent.invoke(...)
    return response['messages'][-1].content  # 只返回最终答案

# 方案2：返回完整对话历史（需要时）
@tool(response_format="content_and_artifact")
def subagent_with_history(request: str):
    response = agent.invoke(...)
    content = response['messages'][-1].content
    artifact = response['messages']  # 完整历史
    return content, artifact
```

#### 6. 监控和调试

```python
# 添加日志追踪SubAgent调用
print("Agent调用链：")
for msg in response['messages']:
    if hasattr(msg, 'tool_calls'):
        for tc in msg.tool_calls:
            print(f"  → {tc['name']}")

# 使用LangSmith追踪完整trace
```

---

### 进阶话题

#### LangGraph实现复杂编排

本章使用`create_agent`简化了实现，但对于更复杂的Multi-Agent系统，推荐使用LangGraph：

```python
from langgraph.graph import StateGraph

# 定义状态
class MultiAgentState(TypedDict):
    messages: list
    current_agent: str
    context: dict

# 构建Graph
graph = StateGraph(MultiAgentState)
graph.add_node("supervisor", supervisor_node)
graph.add_node("agent_a", agent_a_node)
graph.add_node("agent_b", agent_b_node)
graph.add_conditional_edges("supervisor", routing_logic)
# ...
```

优势：
- 可视化工作流
- 支持复杂的条件路由
- State持久化
- Human-in-the-loop支持

---

### 学习资源

1. **LangChain官方教程**：
   - Subagents: Personal Assistant
   - Handoffs: Customer Support
   - Router: Knowledge Base

2. **LangGraph文档**：
   - Multi-Agent with StateGraph
   - Choosing between Graph and Functional APIs

3. **实战参考**：
   - Deep Agents (LangGraph高级Agent框架)
   - Subagents in Deep Agents

---

## 下一步学习

完成Multi-Agent系统学习后，建议进入：
1. **第15章：Persistence基础** - 为Agent添加记忆
2. **LangGraph深入** - 构建复杂的自定义工作流
3. **生产部署** - 监控、优化、扩展Multi-Agent系统